# LIDAR problem

To solve the LIDAR problem depicted in the image, we need to perform a matrix transformation. The image provides a visual representation of the problem, showcasing the input data and the desired output.

The LIDAR problem involves processing data obtained from a LIDAR sensor, which measures distances to objects in its surroundings. The goal is to transform the raw LIDAR data using a matrix operation to obtain a transformed representation of the environment.

In the subsequent sections, we will explore the steps required to solve the LIDAR problem and provide code examples to demonstrate the implementation of the matrix transformation.

![img1](lidar_problem.png)

let us begin transforming the polar coordinates into the cartesian ones

## Point A

In [49]:
import numpy as np 

P_l = [(2.04,0), (2.6, np.deg2rad(20)), (3.39, np.deg2rad(-30))]
cartesian_p = [[x[0]*np.cos(x[1]), x[0]*np.sin(x[1]), 0, 1] for x in P_l]
print(f"Cartesian transformation for each particle {cartesian_p}")



Cartesian transformation for each particle [[2.04, 0.0, 0, 1], [2.443200814043362, 0.8892523726467387, 0, 1], [2.9358261188292474, -1.6949999999999998, 0, 1]]


## Load the Transformation matrix class

In [50]:
import numpy as np

class TransformationMatrix:
  def __init__(self, matrix:np.array=None):
    if matrix is None:
      self.matrix = np.eye(4)
    else:
      self.matrix = matrix
    return

  def transform_deg2rad(self, degree:float) -> float:
    return np.deg2rad(degree)

  def rot_x(self, degree:float) -> np.array:
    deg_rad = self.transform_deg2rad(degree)
    self.matrix = np.dot(np.array([[1, 0, 0, 0],
                            [0, np.cos(deg_rad), -np.sin(deg_rad), 0],
                            [0, np.sin(deg_rad), np.cos(deg_rad), 0],
                            [0, 0, 0, 1]]), self.matrix)
    return self.matrix

  def rot_y(self, degree:float) -> np.array:
    deg_rad = self.transform_deg2rad(degree)
    self.matrix = np.dot(np.array([[np.cos(deg_rad), 0, np.sin(deg_rad), 0],
                            [0, 1, 0, 0],
                            [-np.sin(deg_rad), 0, np.cos(deg_rad), 0],
                            [0, 0, 0, 1]]), self.matrix)
    return self.matrix

  def rot_z(self, degree:float) -> np.array:
    deg_rad = self.transform_deg2rad(degree)
    self.matrix = np.dot(np.array([[np.cos(deg_rad), -np.sin(deg_rad), 0, 0],
                            [np.sin(deg_rad), np.cos(deg_rad), 0, 0],
                            [0, 0, 1, 0],
                            [0, 0, 0, 1]], self.matrix))
    return self.matrix

  def trasl_x(self, length:float) -> np.array:
    self.matrix = np.dot(np.array([[1, 0, 0, length],
                                          [0, 1, 0, 0],
                                          [0, 0, 1, 0],
                                          [0, 0, 0, 1]]), self.matrix)
    return self.matrix

  def trasl_y(self, length:float) -> np.array:
    self.matrix = np.dot(np.array([[1, 0, 0, 0],
                            [0, 1, 0, length],
                            [0, 0, 1, 0],
                            [0, 0, 0, 1]]), self.matrix)
    return self.matrix

  def trasl_z(self, length:float) -> np.array:
    self.matrix = np.dot (np.array([[1, 0, 0, 0],
                            [0, 1, 0, 0],
                            [0, 0, 1, length],
                            [0, 0, 0, 1]]), self.matrix)
    return self.matrix

Now we can make the corresponding transformation from point ${R}$ to point ${L}$, in other words $T^R_L$. After that we apply the following formula:

$$
\begin{equation}
\begin{bmatrix}
p_R \\
1
\end{bmatrix} =
\begin{bmatrix}
R^R_L & t^R_L \\
0 & 1
\end{bmatrix}
\begin{bmatrix}
p_L \\
1
\end{bmatrix}
\end{equation}
$$

In [51]:
phis = [0, 35, 50]
p_R = []
for ang, p in zip(phis, cartesian_p):
    Trl = TransformationMatrix()
    Trl.trasl_y(-0.1)
    Trl.trasl_z(0.3)
    Trl.trasl_x(0.12)
    Trl.rot_y(-ang)
    __result = np.dot(Trl.matrix, p)
    print(f"Result for p={p} is {__result}")
    p_R.append(__result)
    
    
    

Result for p=[2.04, 0.0, 0, 1] is [ 2.16 -0.1   0.3   1.  ]
Result for p=[2.443200814043362, 0.8892523726467387, 0, 1] is [1.92757826 0.78925237 1.7159372  1.        ]
Result for p=[2.9358261188292474, -1.6949999999999998, 0, 1] is [ 1.73443383 -1.795       2.5337349   1.        ]


## Point B

In [52]:
from scipy.spatial.transform import Rotation as R

def create_transformation_matrix(rotation_matrix, translation_vector):
    transformation_matrix = np.eye(4)
    transformation_matrix[:3, :3] = rotation_matrix
    transformation_matrix[:3, 3] = translation_vector
    return transformation_matrix

def quaternion_to_matrix(quaternion):
    r = R.from_quat(quaternion)
    return r.as_matrix()

quaternion = [0.94, 0, 0, 0.34]
matrix = quaternion_to_matrix(quaternion)
trans_i_r = create_transformation_matrix(matrix, [5, -2, -0.2])
print(f"Transformation matrix for quaternion {quaternion} is {trans_i_r}")
    

Transformation matrix for quaternion [0.94, 0, 0, 0.34] is [[ 1.          0.          0.          5.        ]
 [ 0.         -0.76861489 -0.63971177 -2.        ]
 [ 0.          0.63971177 -0.76861489 -0.2       ]
 [ 0.          0.          0.          1.        ]]


Now we obtained the transformation matrix from the inertial system to the Robot, we will apply the particle position from the new transformation matrix.

In [53]:
for p in p_R:
    __result = np.dot(trans_i_r, p)
    print(f"Result for p={p} is {__result}")

Result for p=[ 2.16 -0.1   0.3   1.  ] is [ 7.16       -2.11505204 -0.49455564  1.        ]
Result for p=[1.92757826 0.78925237 1.7159372  1.        ] is [ 6.92757826 -3.70433635 -1.01400086  1.        ]
Result for p=[ 1.73443383 -1.795       2.5337349   1.        ] is [ 6.73443383 -2.24119631 -3.295749    1.        ]
